In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
#cd "/content/drive/MyDrive/data_project3/train"

/content/drive/MyDrive/data_project3/train


In [31]:
#!unzip -qq "/content/drive/MyDrive/data_project3/train/train.zip"

replace 0.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import json
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import datetime
import copy
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch
print(os.listdir())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['.config', 'drive', 'sample_data']


In [3]:
import os
os.chdir('/content/drive/MyDrive/data_project3')
os.getcwd()

'/content/drive/MyDrive/data_project3'

In [ ]:
#!git clone https://github.com/lukemelas/EfficientNet-PyTorch.git

In [4]:
os.chdir('./EfficientNet-PyTorch')
print(os.listdir())

['.git', '.github', '.gitignore', 'LICENSE', 'README.md', 'efficientnet_pytorch', 'examples', 'hubconf.py', 'setup.py', 'sotabench.py', 'sotabench_setup.sh', 'tests', 'tf_to_pytorch', 'efficientnet_pytorch.egg-info']


In [ ]:
#!pip install -e .

In [5]:
os.chdir('/content/drive/MyDrive/data_project3/EfficientNet-PyTorch')

In [40]:
from efficientnet_pytorch import EfficientNet, get_model_params
model = EfficientNet.from_name('efficientnet-b4')
#print(model)

In [41]:
override={'num_classes':12, 'image_size': 5000}
blocks_args, global_params =get_model_params('efficientnet-b4', override)

In [42]:
blocks_args

[BlockArgs(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=2, kernel_size=3, stride=[2], expand_ratio=6, input_filters=16, output_filters=24, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=2, kernel_size=5, stride=[2], expand_ratio=6, input_filters=24, output_filters=40, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=3, kernel_size=3, stride=[2], expand_ratio=6, input_filters=40, output_filters=80, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=3, kernel_size=5, stride=[1], expand_ratio=6, input_filters=80, output_filters=112, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=4, kernel_size=5, stride=[2], expand_ratio=6, input_filters=112, output_filters=192, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=6, input_filters=192, output_filters=320, se_ratio=0.25, id_skip=True)]

### Efficient Model

In [43]:
from torch import nn
from torch.nn import functional as F
class SwishImplementation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))


class MemoryEfficientSwish(nn.Module):
    def forward(self, x):
        return SwishImplementation.apply(x)

In [44]:
import re
import math
import collections
from functools import partial


def get_same_padding_conv1d(image_size=None):
    """Chooses static padding if you have specified an image size, and dynamic padding otherwise.
       Static padding is necessary for ONNX exporting of models.
    Args:
        image_size (int or tuple): Size of the image.
    Returns:
        Conv2dDynamicSamePadding or Conv2dStaticSamePadding.
    """
    return partial(Conv1dStaticSamePadding, image_size=image_size)

class Conv1dStaticSamePadding(nn.Conv1d):
    """2D Convolutions like TensorFlow's 'SAME' mode, with the given input image size.
       The padding mudule is calculated in construction function, then used in forward.
    """

    # With the same calculation as Conv2dDynamicSamePadding

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, image_size=None, **kwargs):
        super().__init__(in_channels, out_channels, kernel_size, stride, **kwargs)
        self.stride = stride if isinstance(stride, int) else stride[0]
        # Calculate padding based on image size and save it
        assert image_size is not None
        ih = image_size if isinstance(image_size, int) else image_size[0]
        kh = self.weight.size()[-1]
        sh = self.stride
        oh = math.ceil(ih / sh)
        self.pad_h = max((oh - 1) * self.stride + (kh - 1) * self.dilation[0] + 1 - ih, 0)

    def forward(self, x):
        if self.pad_h > 0:
          x = F.pad(x,(self.pad_h // 2, self.pad_h - self.pad_h // 2))
        x = F.conv1d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

def calculate_output_image_size(input_image_size, stride):
    """Calculates the output image size when using Conv2dSamePadding with a stride.
       Necessary for static padding. Thanks to mannatsingh for pointing this out.
    Args:
        input_image_size (int, tuple or list): Size of input image.
        stride (int, tuple or list): Conv2d operation's stride.
    Returns:
        output_image_size: A list [H,W].
    """
    if input_image_size is None:
        return None
    image_height =input_image_size
    stride = stride if isinstance(stride, int) else stride[0]
    image_height = int(math.ceil(image_height / stride))
    return image_height

def drop_connect(inputs, p, training):
    """Drop connect.
    Args:
        input (tensor: BCWH): Input of this structure.
        p (float: 0.0~1.0): Probability of drop connection.
        training (bool): The running mode.
    Returns:
        output: Output after drop connection.
    """
    assert 0 <= p <= 1, 'p must be in range of [0,1]'

    if not training:
        return inputs

    batch_size = inputs.shape[0]
    keep_prob = 1 - p

    # generate binary_tensor mask according to probability (p for 0, 1-p for 1)
    random_tensor = keep_prob
    random_tensor += torch.rand([batch_size, 1, 1], dtype=inputs.dtype, device=inputs.device)
    binary_tensor = torch.floor(random_tensor)

    output = inputs / keep_prob * binary_tensor
    return output

def round_filters(filters, global_params):
    """Calculate and round number of filters based on width multiplier.
       Use width_coefficient, depth_divisor and min_depth of global_params.
    Args:
        filters (int): Filters number to be calculated.
        global_params (namedtuple): Global params of the model.
    Returns:
        new_filters: New filters number after calculating.
    """
    multiplier = global_params.width_coefficient
    if not multiplier:
        return filters
    # TODO: modify the params names.
    #       maybe the names (width_divisor,min_width)
    #       are more suitable than (depth_divisor,min_depth).
    divisor = global_params.depth_divisor
    min_depth = global_params.min_depth
    filters *= multiplier
    min_depth = min_depth or divisor  # pay attention to this line when using min_depth
    # follow the formula transferred from official TensorFlow implementation
    new_filters = max(min_depth, int(filters + divisor / 2) // divisor * divisor)
    if new_filters < 0.9 * filters:  # prevent rounding by more than 10%
        new_filters += divisor
    return int(new_filters)

def round_repeats(repeats, global_params):
    """Calculate module's repeat number of a block based on depth multiplier.
       Use depth_coefficient of global_params.
    Args:
        repeats (int): num_repeat to be calculated.
        global_params (namedtuple): Global params of the model.
    Returns:
        new repeat: New repeat number after calculating.
    """
    multiplier = global_params.depth_coefficient
    if not multiplier:
        return repeats
    # follow the formula transferred from official TensorFlow implementation
    return int(math.ceil(multiplier * repeats))

In [45]:
class MBConvBlock(nn.Module):
    """Mobile Inverted Residual Bottleneck Block.
    Args:
        block_args (namedtuple): BlockArgs, defined in utils.py.
        global_params (namedtuple): GlobalParam, defined in utils.py.
        image_size (tuple or list): [image_height, image_width].
    References:
        [1] https://arxiv.org/abs/1704.04861 (MobileNet v1)
        [2] https://arxiv.org/abs/1801.04381 (MobileNet v2)
        [3] https://arxiv.org/abs/1905.02244 (MobileNet v3)
    """

    def __init__(self, block_args, global_params, image_size=None):
        super().__init__()
        self._block_args = block_args
        self._bn_mom = 1 - global_params.batch_norm_momentum  # pytorch's difference from tensorflow
        self._bn_eps = global_params.batch_norm_epsilon
        self.has_se = (self._block_args.se_ratio is not None) and (0 < self._block_args.se_ratio <= 1)
        self.id_skip = block_args.id_skip  # whether to use skip connection and drop connect

        # Expansion phase (Inverted Bottleneck)
        inp = self._block_args.input_filters  # number of input channels
        oup = self._block_args.input_filters * self._block_args.expand_ratio  # number of output channels
        if self._block_args.expand_ratio != 1:
            Conv1d = get_same_padding_conv1d(image_size=image_size)
            self._expand_conv = Conv1d(in_channels=inp, out_channels=oup, kernel_size=1, bias=False)
            self._bn0 = nn.BatchNorm1d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)
            # image_size = calculate_output_image_size(image_size, 1) <-- this wouldn't modify image_size

        # Depthwise convolution phase
        k = self._block_args.kernel_size
        s = self._block_args.stride
        Conv1d = get_same_padding_conv1d(image_size=image_size)
        self._depthwise_conv = Conv1d(
            in_channels=oup, out_channels=oup, groups=oup,  # groups makes it depthwise
            kernel_size=k, stride=s, bias=False)
        self._bn1 = nn.BatchNorm1d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)
        image_size = calculate_output_image_size(image_size, s)

        # Squeeze and Excitation layer, if desired
        if self.has_se:
            Conv1d = get_same_padding_conv1d(image_size=(1, 1))
            num_squeezed_channels = max(1, int(self._block_args.input_filters * self._block_args.se_ratio))
            self._se_reduce = Conv1d(in_channels=oup, out_channels=num_squeezed_channels, kernel_size=1)
            self._se_expand = Conv1d(in_channels=num_squeezed_channels, out_channels=oup, kernel_size=1)

        # Pointwise convolution phase
        final_oup = self._block_args.output_filters
        Conv1d = get_same_padding_conv1d(image_size=image_size)
        self._project_conv = Conv1d(in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False)
        self._bn2 = nn.BatchNorm1d(num_features=final_oup, momentum=self._bn_mom, eps=self._bn_eps)
        self._swish = MemoryEfficientSwish()

    def forward(self, inputs, drop_connect_rate=None):
        """MBConvBlock's forward function.
        Args:
            inputs (tensor): Input tensor.
            drop_connect_rate (bool): Drop connect rate (float, between 0 and 1).
        Returns:
            Output of this block after processing.
        """

        # Expansion and Depthwise Convolution
        x = inputs
        if self._block_args.expand_ratio != 1:
            x = self._expand_conv(inputs)
            x = self._bn0(x)
            x = self._swish(x)

        x = self._depthwise_conv(x)
        x = self._bn1(x)
        x = self._swish(x)

        # Squeeze and Excitation
        if self.has_se:
            x_squeezed = F.adaptive_avg_pool1d(x, 1)
            x_squeezed = self._se_reduce(x_squeezed)
            x_squeezed = self._swish(x_squeezed)
            x_squeezed = self._se_expand(x_squeezed)
            x = torch.sigmoid(x_squeezed) * x

        # Pointwise Convolution
        x = self._project_conv(x)
        x = self._bn2(x)

        # Skip connection and drop connect
        input_filters, output_filters = self._block_args.input_filters, self._block_args.output_filters
        if self.id_skip and self._block_args.stride == 1 and input_filters == output_filters:
            # The combination of skip connection and drop connect brings about stochastic depth.
            if drop_connect_rate:
                x = drop_connect(x, p=drop_connect_rate, training=self.training)
            x = x + inputs  # skip connection
        return x


In [46]:
class EfficientNet(nn.Module):
    """EfficientNet model.
       Most easily loaded with the .from_name or .from_pretrained methods.
    Args:
        blocks_args (list[namedtuple]): A list of BlockArgs to construct blocks.
        global_params (namedtuple): A set of GlobalParams shared between blocks.
    References:
        [1] https://arxiv.org/abs/1905.11946 (EfficientNet)
    Example:
        >>> import torch
        >>> from efficientnet.model import EfficientNet
        >>> inputs = torch.rand(1, 3, 224, 224)
        >>> model = EfficientNet.from_pretrained('efficientnet-b0')
        >>> model.eval()
        >>> outputs = model(inputs)
    """

    def __init__(self, blocks_args=None, global_params=None):
        super(EfficientNet, self).__init__()
        assert isinstance(blocks_args, list), 'blocks_args should be a list'
        assert len(blocks_args) > 0, 'block args must be greater than 0'
        self._global_params = global_params
        self._blocks_args = blocks_args

        # Batch norm parameters
        bn_mom = 1 - self._global_params.batch_norm_momentum
        bn_eps = self._global_params.batch_norm_epsilon

        # Get stem static or dynamic convolution depending on image size
        image_size = global_params.image_size
        Conv1d = get_same_padding_conv1d(image_size=image_size)

        # Stem
        in_channels = 2  # rgb
        out_channels = round_filters(32, self._global_params)  # number of output channels
        self._conv_stem = Conv1d(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        self._bn0 = nn.BatchNorm1d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)
        image_size = calculate_output_image_size(image_size, 2)

        # Build blocks
        self._blocks = nn.ModuleList([])
        for block_args in self._blocks_args:

            # Update block input and output filters based on depth multiplier.
            block_args = block_args._replace(
                input_filters=round_filters(block_args.input_filters, self._global_params),
                output_filters=round_filters(block_args.output_filters, self._global_params),
                num_repeat=round_repeats(block_args.num_repeat, self._global_params)
            )

            # The first block needs to take care of stride and filter size increase.
            self._blocks.append(MBConvBlock(block_args, self._global_params, image_size=image_size))
            image_size = calculate_output_image_size(image_size, block_args.stride)
            if block_args.num_repeat > 1:  # modify block_args to keep same output size
                block_args = block_args._replace(input_filters=block_args.output_filters, stride=1)
            for _ in range(block_args.num_repeat - 1):
                self._blocks.append(MBConvBlock(block_args, self._global_params, image_size=image_size))
                # image_size = calculate_output_image_size(image_size, block_args.stride)  # stride = 1

        # Head
        in_channels = block_args.output_filters  # output of final block
        out_channels = round_filters(1280, self._global_params)
        Conv1d = get_same_padding_conv1d(image_size=image_size)
        self._conv_head = Conv1d(in_channels, out_channels, kernel_size=1, bias=False)
        self._bn1 = nn.BatchNorm1d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Final linear layer
        self._avg_pooling = nn.AdaptiveAvgPool1d(7)
        if self._global_params.include_top:
            self._dropout = nn.Dropout(self._global_params.dropout_rate)
            self._fc = nn.Linear(out_channels*7, self._global_params.num_classes)

        # set activation to memory efficient 
        # by default
        self._swish = MemoryEfficientSwish()
        # set activation to memory efficient swish by default
    def set_swish(self, memory_efficient=True):
        """Sets swish function as memory efficient (for training) or standard (for export).
        Args:
            memory_efficient (bool): Whether to use memory-efficient version of swish.
        """
        self._swish = MemoryEfficientSwish() if memory_efficient else Swish()
        for block in self._blocks:
            block.set_swish(memory_efficient)

    def extract_endpoints(self, inputs):
        """Use convolution layer to extract features
        from reduction levels i in [1, 2, 3, 4, 5].
        Args:
            inputs (tensor): Input tensor.
        Returns:
            Dictionary of last intermediate features
            with reduction levels i in [1, 2, 3, 4, 5].
            Example:
                >>> import torch
                >>> from efficientnet.model import EfficientNet
                >>> inputs = torch.rand(1, 3, 224, 224)
                >>> model = EfficientNet.from_pretrained('efficientnet-b0')
                >>> endpoints = model.extract_endpoints(inputs)
                >>> print(endpoints['reduction_1'].shape)  # torch.Size([1, 16, 112, 112])
                >>> print(endpoints['reduction_2'].shape)  # torch.Size([1, 24, 56, 56])
                >>> print(endpoints['reduction_3'].shape)  # torch.Size([1, 40, 28, 28])
                >>> print(endpoints['reduction_4'].shape)  # torch.Size([1, 112, 14, 14])
                >>> print(endpoints['reduction_5'].shape)  # torch.Size([1, 320, 7, 7])
                >>> print(endpoints['reduction_6'].shape)  # torch.Size([1, 1280, 7, 7])
        """
        endpoints = dict()

        # Stem
        x = self._swish(self._bn0(self._conv_stem(inputs)))
        prev_x = x

        # Blocks
        for idx, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self._blocks)  # scale drop connect_rate
            x = block(x, drop_connect_rate=drop_connect_rate)
            if prev_x.size(2) > x.size(2):
                endpoints['reduction_{}'.format(len(endpoints) + 1)] = prev_x
            elif idx == len(self._blocks) - 1:
                endpoints['reduction_{}'.format(len(endpoints) + 1)] = x
            prev_x = x

        # Head
        x = self._swish(self._bn1(self._conv_head(x)))
        endpoints['reduction_{}'.format(len(endpoints) + 1)] = x

        return endpoints

    def extract_features(self, inputs):
        """use convolution layer to extract feature .
        Args:
            inputs (tensor): Input tensor.
        Returns:
            Output of the final convolution
            layer in the efficientnet model.
        """
        # Stem
        x = self._swish(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for idx, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self._blocks)  # scale drop connect_rate
            x = block(x, drop_connect_rate=drop_connect_rate)

        # Head
        x = self._swish(self._bn1(self._conv_head(x)))

        return x

    def forward(self, inputs):
        """EfficientNet's forward function.
           Calls extract_features to extract features, applies final linear layer, and returns logits.
        Args:
            inputs (tensor): Input tensor.
        Returns:
            Output of this model after processing.
        """
        # Convolution layers
        x = self.extract_features(inputs)
        # Pooling and final linear layer
        x = self._avg_pooling(x)
        if self._global_params.include_top:
            x = x.flatten(start_dim=1)
            x = self._dropout(x)
            x = self._fc(x)
        return x

In [47]:
model=EfficientNet(blocks_args, global_params)

In [48]:
#print(model)

In [49]:
input=torch.rand((2,2,5000))
output=model(input)
output

tensor([[ 0.2909,  0.0997,  0.2718,  0.1190, -0.1312, -0.3882, -0.1366,  0.1344,
         -0.3354,  0.1046, -0.4847, -0.3504],
        [-0.0604,  0.1397,  0.0334, -0.1573,  0.0213, -0.0520, -0.1006,  0.1978,
         -0.0424,  0.0417,  0.0256,  0.0061]], grad_fn=<AddmmBackward>)

### Helper functions

In [50]:
def extract_age(info_file):
    '''
        info file(###.txt)로부터 나이 정보를 뽑아냅니다.
    '''
    with open(info_file, 'r') as f:
        info = f.read()
        for i, line in enumerate(info.split("\n")):
            if line.startswith("#Age"):
                age = float(line.split(": ")[1].strip())
    return age

def extract_sex(info_file):
    '''
        info file(###.txt)로부터 성별 정보를 뽑아냅니다.
    '''
    with open(info_file, 'r') as f:
            info = f.read()
            for i, line in enumerate(info.split("\n")):
                if line.startswith("#Sex"):
                    sex = line.split(": ")[1].strip()
    return sex


def extract_labels(info_file):
    '''
        info file(###.txt)로부터 label(들) 정보를 뽑아냅니다.
    '''
    with open(info_file, 'r') as f:
            info = f.read()
            for i, line in enumerate(info.split("\n")):
                if line.startswith("#Dx"):
                    labels = line.split(": ")[1].strip()
                    labels = labels.split()
    return labels

def read_files(data_directory, is_training=True):
    '''
        data directory(train 또는 test)로부터 모든 sample들의
        id, age, sex, recording, labels 정보를 읽어들여
        (id, age, sex, recording, labels)의 list를 반환합니다.
        is_training=False일 경우엔 labels 정보를 읽어들이지 않습니다.
    '''
    list_id = []
    list_age = []
    list_sex = []
    list_recording = []
    list_labels = []
    for f in os.listdir(data_directory):
        root, extension = os.path.splitext(f)
        if not root.startswith(".") and extension == ".txt":
            list_id.append(int(root))
            info_file = os.path.join(data_directory, root + ".txt")
            recording_file = os.path.join(data_directory, root + ".npy")
            age = extract_age(info_file)
            list_age.append(age)
            sex = extract_sex(info_file)
            list_sex.append(sex)
            with open(recording_file, 'rb') as g:
                recording = np.load(g)
                list_recording.append(recording)
            if is_training:
                labels = extract_labels(info_file)
                list_labels.append(labels)
    if is_training:
        return list(zip(list_id, list_age, list_sex, list_recording, list_labels))
    else:
        return list(zip(list_id, list_age, list_sex, list_recording))

### PyTorch Custom Dataset
training sample을 batch 단위로 처리할 수 있도록 torch.uitls.data.Dataset을 이용한 custom dataset을 만들어 줍니다.

In [51]:
class Dataset_ECG(torch.utils.data.Dataset):
    """
        Build ECG dataset
    """
    def __init__(self, dataset, num_classes=12):
        """
            dataset을 읽어들여 id, age, sex, recording, labels를 저장한 list를 만들어 줍니다.
        """
        self.sample_id = []
        self.sample_age = []
        self.sample_sex = []
        self.sample_recording = []
        self.sample_labels = []
        self.num_samples = len(dataset)
        
        for idx in range(self.num_samples):
            _id, _age, _sex, _recording, _labels = dataset[idx]
            # model에 input으로 들어가는 data는 torch.Tensor 타입으로 변환해 줍니다.
            age = torch.tensor(_age//10)
            sex = torch.tensor([0,1]) if _sex == "F" else torch.tensor([1,0])
            recording = torch.tensor(_recording)
            labels = torch.tensor(np.zeros(num_classes))
            for label in _labels:
                labels[int(label)] = 1

            self.sample_id.append(_id)
            self.sample_age.append(age)
            self.sample_sex.append(sex)
            self.sample_recording.append(recording)
            self.sample_labels.append(labels)

        print(f'Loaded {self.num_samples} samples...')

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return {
            "id": self.sample_id[idx],
            "age": self.sample_age[idx],
            "sex": self.sample_sex[idx],
            "recording": self.sample_recording[idx],
            "labels": self.sample_labels[idx]
        }

### PyTorch CNN model
간단한 CNN model을 만들어 보겠습니다.

### Setup

In [52]:
#print(os.getcwd())
os.chdir('/content/drive/MyDrive/data_project3')

In [53]:
# cuda gpu를 사용할 수 있을 경우 사용합니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [54]:
# Training에 사용될 hyperparameter를 정해줍니다.
EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 0.001

In [55]:
training_dir = '/content/drive/MyDrive/data_project3/train'
test_dir = '/content/drive/MyDrive/data_project3/test'

In [56]:
#training_dataset = torch.load('augmented_train_2.pt')
training_data = sorted(read_files(training_dir), key=lambda sample: sample[0])
#validation_set = torch.load('val.pt')
num_training = len(training_data)

KeyboardInterrupt: ignored

In [57]:
training_dataset = Dataset_ECG(training_data, num_classes=12)

Loaded 19212 samples...


In [58]:
# Training dataset을 batch 단위로 읽어들일 수 있도록 DataLoader를 만들어줍니다.
training_loader = torch.utils.data.DataLoader(training_dataset,pin_memory=True, batch_size=BATCH_SIZE)

### Training

In [59]:
model = model.to(device)
model.to(device)
model.train()
pos=[811, 1062, 958, 3204, 574, 515, 625, 1203, 10258, 1114, 2451, 563]
p=[15370-x for x in pos]
p_w=[x/y for x, y in zip(p, pos)]
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor(p_w).to(device)) # for multi-label classification
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [25]:
# https://dacon.io/codeshare/2377?dtype=recent&s_id=0 코드 참조
!pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git

  Cloning https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to /tmp/pip-req-build-yacw49k4
  Running command git clone -q https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git /tmp/pip-req-build-yacw49k4
  Created wheel for torch-poly-lr-decay: filename=torch_poly_lr_decay-0.0.1-cp37-none-any.whl size=3847 sha256=efa50c4b903c3391137b501b66313f6c78f2f31daf3cf2f65baa639ab9580baa
  Stored in directory: /tmp/pip-ephem-wheel-cache-cl8l2oas/wheels/5a/b7/09/d748b20c9bdfc768a33c37a28b2ad7dd9ec3e79e5152cb1618
Successfully built torch-poly-lr-decay


In [60]:
from torch_poly_lr_decay import PolynomialLRDecay

decay_steps = (num_training//BATCH_SIZE)*EPOCHS
scheduler_poly_lr_decay = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)

In [61]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
import csv

def cal_run_time(sec):
    times = str(datetime.timedelta(seconds=sec)).split(".")
    return times[0]


def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    train_start = time.time()

    #best_model_wts = copy.deepcopy(model.state_dict())
    best_f1_score = 0.0
    epoch_loss, epoch_score = [], []
    # Each epoch has a training and validation phase
    for epoch in range(1, num_epochs+1):
        print('***** Epoch {}/{} *****'.format(epoch, num_epochs))
        start = time.time()

        ## Train
        model.train()
        train_loss_list = []
        epoch_training_loss_sum = 0.0
        for i_batch, sample_batched in enumerate(training_loader):
            b_recording = sample_batched["recording"].to(device)       
            #b_sex=sample_batched["sex"].to(device)
            #b_age=sample_batched["age"].to(device)
            b_labels = sample_batched["labels"].to(device)          
            optimizer.zero_grad()
            b_out = model(b_recording.float())                      
            loss = criterion(b_out, b_labels)
            loss.backward()
            optimizer.step()
            epoch_training_loss_sum += loss.item() * b_labels.shape[0]
            scheduler.step()
        
        # batch train loss 기록 
        train_loss = epoch_training_loss_sum / num_training
        epoch_loss.append(train_loss)

        # calculate time
        times = cal_run_time(time.time()-start)
        print('train_loss : {:.5f}\ttime : {}\n'.format(train_loss, times))

        # epoch 5 단위마다 model 저장
        if epoch%5==0:
            torch.save(model,f'eff_model_{epoch}.pt')


        '''
        ## Validation
        model.eval()
        validation_prediction_df = pd.DataFrame(columns=['labels'])
        validation_prediction_df.index.name = 'id'
        validation_true_labels_df = pd.DataFrame(columns=['labels'])
        validation_true_labels_df.index.name = 'id'

        with torch.no_grad():
            for idx in range(len(validation_set)):
                validation_sample = validation_set[idx]
                _, _, _, recording, labels = validation_sample  
                out = model(torch.tensor(recording).unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
                sample_prediction = torch.nn.functional.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
                indices_of_1s = np.where(sample_prediction.cpu())[0]
                str_indices_of_1s = ' '.join(map(str, indices_of_1s))
                validation_prediction_df.loc[idx] = [str_indices_of_1s]
                        
                str_true_labels = ' '.join(labels)
                validation_true_labels_df.loc[idx] = [str_true_labels]
  
        mlb = MultiLabelBinarizer(classes=['0','1','2','3','4','5','6','7','8','9','10','11'])
        mlb.fit(map(str.split, validation_true_labels_df['labels'].values))

        macro_f1_validation = f1_score(mlb.transform(map(str.split, validation_true_labels_df['labels'].values)), mlb.transform(map(str.split, validation_prediction_df['labels'].values)), average='macro')
        epoch_score.append(macro_f1_validation)

        # calculate time
        times = cal_run_time(time.time()-start())

        print('train_loss : {:.5f}\tvalid_f1_score : {:.5f}\ttime : {}\n'.format(train_loss, macro_f1_validation, times))


        # deep copy the model & save best model
        if macro_f1_validation > best_f1_score:
            best_idx = epoch
            best_f1_score = macro_f1_validation
            #best_model_wts = copy.deepcopy(model.state_dict())
            #torch.save(model.state_dict(), f'eff2_best_model.pt')
            torch.save(model, f'eff2_best_model.pt')
            print('==> best model saved - epoch: %d / f1_score: %.5f\n'%(best_idx, best_f1_score))
        

    times = cal_run_time(time.time()-train_start())
    print('Training complete in {}'.format(times))
    print('Best valid f1 score: %d - %.5f' %(best_idx, best_f1_score))
    '''

    # calculate time
    times = cal_run_time(time.time()-train_start)
    print('Training completed in {}'.format(times))

    # save epoch_loss & epoch_score
    with open('epoch_data', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(epoch_loss)
        #writer.writerow(epoch_score)
    print('\nepoch data saved.')

    #return model, best_idx, best_f1_score, epoch_loss, epoch_score, validation_prediction_df, validation_true_labels_df


In [ ]:
# Train start
train_model(model, criterion, optimizer, scheduler_poly_lr_decay, EPOCHS)

train_loss : 0.56028	time : 0:12:41

***** Epoch 12/50 *****
train_loss : 0.54357	time : 0:12:45

***** Epoch 13/50 *****
train_loss : 0.52626	time : 0:12:45

***** Epoch 14/50 *****
train_loss : 0.51069	time : 0:12:45

***** Epoch 15/50 *****
train_loss : 0.49538	time : 0:12:45

***** Epoch 16/50 *****
train_loss : 0.48351	time : 0:12:44

***** Epoch 17/50 *****
train_loss : 0.46373	time : 0:12:43

***** Epoch 18/50 *****
train_loss : 0.44974	time : 0:12:43

***** Epoch 19/50 *****
train_loss : 0.43043	time : 0:12:42

***** Epoch 20/50 *****
train_loss : 0.41024	time : 0:12:43

***** Epoch 21/50 *****
train_loss : 0.38939	time : 0:12:43

***** Epoch 22/50 *****
train_loss : 0.37320	time : 0:12:42

***** Epoch 23/50 *****
train_loss : 0.34953	time : 0:12:42

***** Epoch 24/50 *****
train_loss : 0.32954	time : 0:12:42

***** Epoch 25/50 *****
train_loss : 0.31232	time : 0:12:42

***** Epoch 26/50 *****
train_loss : 0.29706	time : 0:12:42

***** Epoch 27/50 *****


### Evaluation
evalutate on validation set

In [ ]:
'''
name=f'eff_best_model.pt'
model=torch.load(name)
model.to(device)

model.eval()


validation_prediction_df = pd.DataFrame(columns=['labels'])
validation_prediction_df.index.name = 'id'
validation_true_labels_df = pd.DataFrame(columns=['labels'])
validation_true_labels_df.index.name = 'id'

with torch.no_grad():
    for idx in range(len(validation_set)):
        validation_sample = validation_set[idx]
        _, _, _, recording, labels = validation_sample
        out = model(torch.tensor(recording).unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
        sample_prediction = torch.nn.functional.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
        indices_of_1s = np.where(sample_prediction.cpu())[0]
        str_indices_of_1s = ' '.join(map(str, indices_of_1s))
        validation_prediction_df.loc[idx] = [str_indices_of_1s]
        
        str_true_labels = ' '.join(labels)
        validation_true_labels_df.loc[idx] = [str_true_labels]
'''

In [ ]:
#print(validation_prediction_df[:10])

In [ ]:
#print(validation_true_labels_df[:10])

### Test Prediction
학습된 모델로 test_set에 대한 prediction을 진행합니다.

In [ ]:
model.eval()

test_prediction_df = pd.DataFrame(columns=['labels'])
test_prediction_df.index.name = 'id'

with torch.no_grad():
    for idx in range(len(test_set)):
        test_sample = test_set[idx]
        _, _, _, recording = test_sample
        out = model(torch.tensor(recording).unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
        sample_prediction = torch.nn.functional.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
        indices_of_1s = np.where(sample_prediction.cpu())[0]
        str_indices_of_1s = ' '.join(map(str, indices_of_1s))
        test_prediction_df.loc[idx] = [str_indices_of_1s]

In [ ]:
test_prediction_df[:10]

In [ ]:
test_prediction_df.to_csv('my_submission.csv')